# Manipulating and preparing data for Modelling

In [2]:
from pyspark import SparkContext
sc=SparkContext(master='local[2]',
               appName='my-spark')
from pyspark.sql import SparkSession
spark=SparkSession(sparkContext=sc)

## Dense vs Sparse Vector

A vector can be represented in dense and sparse formats. A dense vector is a regular vector that has each elements printed. A sparse vector use three components to represent a vector but with less memory.

In [2]:
from pyspark.ml.linalg import Vector, DenseVector, SparseVector

#### Dense Vector

In [3]:
dv=DenseVector([1,0,0,0,4.5,0])
dv

DenseVector([1.0, 0.0, 0.0, 0.0, 4.5, 0.0])

#### Sparse Vector 


In [5]:
sv=SparseVector(6,{0:1.0,4:4.5})
sv

SparseVector(6, {0: 1.0, 4: 4.5})

#### Convert sparse vector to dense vector

In [6]:
DenseVector(sv.toArray())

DenseVector([1.0, 0.0, 0.0, 0.0, 4.5, 0.0])

#### Convert dense vector to sparse vector

## Manipulating DataFrame

### Selecting Rows

#### Select Rows by index


In [12]:
mtcars=spark.read.csv('mtcars.csv',header=True,inferSchema=True)
mtcars.dtypes
mtcars=mtcars.withColumnRenamed('_c0','model')
mtcars.show(4)

+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|         model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|     Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
| Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|    Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 4 rows



In [13]:
mtcars.rdd.zipWithIndex().take(5)

[(Row(model='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4),
  0),
 (Row(model='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4),
  1),
 (Row(model='Datsun 710', mpg=22.8, cyl=4, disp=108.0, hp=93, drat=3.85, wt=2.32, qsec=18.61, vs=1, am=1, gear=4, carb=1),
  2),
 (Row(model='Hornet 4 Drive', mpg=21.4, cyl=6, disp=258.0, hp=110, drat=3.08, wt=3.215, qsec=19.44, vs=1, am=0, gear=3, carb=1),
  3),
 (Row(model='Hornet Sportabout', mpg=18.7, cyl=8, disp=360.0, hp=175, drat=3.15, wt=3.44, qsec=17.02, vs=0, am=0, gear=3, carb=2),
  4)]

In [14]:
from pyspark.sql import Row
mtcars.rdd.zipWithIndex().map(lambda x: x[0]+Row(index=x[1])).take(5)

[('Mazda RX4', 21.0, 6, 160.0, 110, 3.9, 2.62, 16.46, 0, 1, 4, 4, 0),
 ('Mazda RX4 Wag', 21.0, 6, 160.0, 110, 3.9, 2.875, 17.02, 0, 1, 4, 4, 1),
 ('Datsun 710', 22.8, 4, 108.0, 93, 3.85, 2.32, 18.61, 1, 1, 4, 1, 2),
 ('Hornet 4 Drive', 21.4, 6, 258.0, 110, 3.08, 3.215, 19.44, 1, 0, 3, 1, 3),
 ('Hornet Sportabout', 18.7, 8, 360.0, 175, 3.15, 3.44, 17.02, 0, 0, 3, 2, 4)]

In [16]:
col_names=mtcars.columns+['index']

In [17]:
mtcars.rdd.zipWithIndex().map(lambda x: x[0]+Row(index=x[1])).toDF(col_names).show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|    0|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|    1|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|    2|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|    3|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|    4|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|    5|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|    6|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|    7|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
|           Merc

In [20]:
mtcars_new=mtcars.rdd.zipWithIndex().map(lambda x: x[0]+Row(index=x[1])).toDF(col_names)
mtcars_new.show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|    0|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|    1|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|    2|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|    3|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|    4|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|    5|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|    6|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|    7|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
|           Merc

In [21]:
mtcars_new.filter(mtcars_new.index.isin(1,2,3,8)).show()

+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|         model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
| Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|    1|
|    Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|    2|
|Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|    3|
|      Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+



In [22]:
mtcars_new.filter(mtcars_new.index.between(2,8)).show()

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|            model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|    2|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|    3|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|    4|
|          Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|    5|
|       Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|    6|
|        Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|    7|
|         Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+



In [23]:
mtcars_new.filter(mtcars_new.index>4).show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|    5|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|    6|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|    7|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
|           Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|    9|
|          Merc 280C|17.8|  6|167.6|123|3.92| 3.44| 18.9|  1|  0|   4|   4|   10|
|         Merc 450SE|16.4|  8|275.8|180|3.07| 4.07| 17.4|  0|  0|   3|   3|   11|
|         Merc 450SL|17.3|  8|275.8|180|3.07| 3.73| 17.6|  0|  0|   3|   3|   12|
|        Merc 450SLC|15.2|  8|275.8|180|3.07| 3.78| 18.0|  0|  0|   3|   3|   13|
| Cadillac Fleet

In [26]:
new=mtcars_new.filter(mtcars_new.cyl==4)
new.show()

+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|         model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|index|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+-----+
|    Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|    2|
|     Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|    7|
|      Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|    8|
|      Fiat 128|32.4|  4| 78.7| 66|4.08|  2.2|19.47|  1|  1|   4|   1|   17|
|   Honda Civic|30.4|  4| 75.7| 52|4.93|1.615|18.52|  1|  1|   4|   2|   18|
|Toyota Corolla|33.9|  4| 71.1| 65|4.22|1.835| 19.9|  1|  1|   4|   1|   19|
| Toyota Corona|21.5|  4|120.1| 97| 3.7|2.465|20.01|  1|  0|   3|   1|   20|
|     Fiat X1-9|27.3|  4| 79.0| 66|4.08|1.935| 18.9|  1|  1|   4|   1|   25|
| Porsche 914-2|26.0|  4|120.3| 91|4.43| 2.14| 16.7|  0|  1|   5|   2|   26|
|  Lotus Europa|30.4|  4| 95.1|113|3.77|1.513| 16.9|  1|  1|   5|   2|   27|

In [31]:
original_columns_all=[eval('mtcars.' + x) for x in mtcars.columns]
original_columns_all.

[Column<b'model'>,
 Column<b'mpg'>,
 Column<b'cyl'>,
 Column<b'disp'>,
 Column<b'hp'>,
 Column<b'drat'>,
 Column<b'wt'>,
 Column<b'qsec'>,
 Column<b'vs'>,
 Column<b'am'>,
 Column<b'gear'>,
 Column<b'carb'>]

In [33]:
from pyspark.sql import functions as F
filter_column=F.when((mtcars.vs==1) & (mtcars.am==1),1).name('filter')

In [34]:
original_columns_all + [filter_column]

[Column<b'model'>,
 Column<b'mpg'>,
 Column<b'cyl'>,
 Column<b'disp'>,
 Column<b'hp'>,
 Column<b'drat'>,
 Column<b'wt'>,
 Column<b'qsec'>,
 Column<b'vs'>,
 Column<b'am'>,
 Column<b'gear'>,
 Column<b'carb'>,
 Column<b'CASE WHEN ((vs = 1) AND (am = 1)) THEN 1 END AS `filter`'>]

In [35]:
mtcars_filter=mtcars.select(original_columns_all + [filter_column])
mtcars_filter.show(20)

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+------+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|filter|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+------+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|  null|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|  null|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|     1|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|  null|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|  null|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|  null|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|  null|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2|  null|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|  null|
|   

In [38]:
mtcars_filter.filter(mtcars_filter['filter']==1).show(20)

+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+------+
|         model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|filter|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+------+
|    Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|     1|
|      Fiat 128|32.4|  4| 78.7| 66|4.08|  2.2|19.47|  1|  1|   4|   1|     1|
|   Honda Civic|30.4|  4| 75.7| 52|4.93|1.615|18.52|  1|  1|   4|   2|     1|
|Toyota Corolla|33.9|  4| 71.1| 65|4.22|1.835| 19.9|  1|  1|   4|   1|     1|
|     Fiat X1-9|27.3|  4| 79.0| 66|4.08|1.935| 18.9|  1|  1|   4|   1|     1|
|  Lotus Europa|30.4|  4| 95.1|113|3.77|1.513| 16.9|  1|  1|   5|   2|     1|
|    Volvo 142E|21.4|  4|121.0|109|4.11| 2.78| 18.6|  1|  1|   4|   2|     1|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+------+



In [39]:
mtcars_filter.filter(mtcars_filter['filter']==1).drop('filter').show(20)

+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|         model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|    Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|      Fiat 128|32.4|  4| 78.7| 66|4.08|  2.2|19.47|  1|  1|   4|   1|
|   Honda Civic|30.4|  4| 75.7| 52|4.93|1.615|18.52|  1|  1|   4|   2|
|Toyota Corolla|33.9|  4| 71.1| 65|4.22|1.835| 19.9|  1|  1|   4|   1|
|     Fiat X1-9|27.3|  4| 79.0| 66|4.08|1.935| 18.9|  1|  1|   4|   1|
|  Lotus Europa|30.4|  4| 95.1|113|3.77|1.513| 16.9|  1|  1|   5|   2|
|    Volvo 142E|21.4|  4|121.0|109|4.11| 2.78| 18.6|  1|  1|   4|   2|
+--------------+----+---+-----+---+----+-----+-----+---+---+----+----+



#### Subset Rows by logical condition

### Selecting Columns

#### Select columns by names
Selecting columns by names is very simple. Just use the `select()` function.

In [41]:
mtcars.select('mpg','cyl').show()

+----+---+
| mpg|cyl|
+----+---+
|21.0|  6|
|21.0|  6|
|22.8|  4|
|21.4|  6|
|18.7|  8|
|18.1|  6|
|14.3|  8|
|24.4|  4|
|22.8|  4|
|19.2|  6|
|17.8|  6|
|16.4|  8|
|17.3|  8|
|15.2|  8|
|10.4|  8|
|10.4|  8|
|14.7|  8|
|32.4|  4|
|30.4|  4|
|33.9|  4|
+----+---+
only showing top 20 rows



In [42]:
mtcars.select([eval('mtcars.'+x) for x in ['model','mpg','cyl']] ).show()

+-------------------+----+---+
|              model| mpg|cyl|
+-------------------+----+---+
|          Mazda RX4|21.0|  6|
|      Mazda RX4 Wag|21.0|  6|
|         Datsun 710|22.8|  4|
|     Hornet 4 Drive|21.4|  6|
|  Hornet Sportabout|18.7|  8|
|            Valiant|18.1|  6|
|         Duster 360|14.3|  8|
|          Merc 240D|24.4|  4|
|           Merc 230|22.8|  4|
|           Merc 280|19.2|  6|
|          Merc 280C|17.8|  6|
|         Merc 450SE|16.4|  8|
|         Merc 450SL|17.3|  8|
|        Merc 450SLC|15.2|  8|
| Cadillac Fleetwood|10.4|  8|
|Lincoln Continental|10.4|  8|
|  Chrysler Imperial|14.7|  8|
|           Fiat 128|32.4|  4|
|        Honda Civic|30.4|  4|
|     Toyota Corolla|33.9|  4|
+-------------------+----+---+
only showing top 20 rows



#### Select columns by index

In [44]:
column_indices=[0,1,3]+list(range(5,9))
column_indices


[0, 1, 3, 5, 6, 7, 8]

In [45]:
[mtcars.columns[x] for x in column_indices]

['model', 'mpg', 'disp', 'drat', 'wt', 'qsec', 'vs']

In [46]:
mtcars.select([eval('mtcars.'+x) for x in [mtcars.columns[x] for x in column_indices]]).show()

+-------------------+----+-----+----+-----+-----+---+
|              model| mpg| disp|drat|   wt| qsec| vs|
+-------------------+----+-----+----+-----+-----+---+
|          Mazda RX4|21.0|160.0| 3.9| 2.62|16.46|  0|
|      Mazda RX4 Wag|21.0|160.0| 3.9|2.875|17.02|  0|
|         Datsun 710|22.8|108.0|3.85| 2.32|18.61|  1|
|     Hornet 4 Drive|21.4|258.0|3.08|3.215|19.44|  1|
|  Hornet Sportabout|18.7|360.0|3.15| 3.44|17.02|  0|
|            Valiant|18.1|225.0|2.76| 3.46|20.22|  1|
|         Duster 360|14.3|360.0|3.21| 3.57|15.84|  0|
|          Merc 240D|24.4|146.7|3.69| 3.19| 20.0|  1|
|           Merc 230|22.8|140.8|3.92| 3.15| 22.9|  1|
|           Merc 280|19.2|167.6|3.92| 3.44| 18.3|  1|
|          Merc 280C|17.8|167.6|3.92| 3.44| 18.9|  1|
|         Merc 450SE|16.4|275.8|3.07| 4.07| 17.4|  0|
|         Merc 450SL|17.3|275.8|3.07| 3.73| 17.6|  0|
|        Merc 450SLC|15.2|275.8|3.07| 3.78| 18.0|  0|
| Cadillac Fleetwood|10.4|472.0|2.93| 5.25|17.98|  0|
|Lincoln Continental|10.4|46

#### Select by columns names that match a regex pattern

In [47]:
import re

In [49]:
sel_col=[x for x in mtcars.columns if re.compile('^d').match(x) is not None]
sel_col

['disp', 'drat']

In [50]:
[eval('mtcars.'+x) for x in sel_col]

[Column<b'disp'>, Column<b'drat'>]

In [57]:
mtcars.select([eval('mtcars.'+x) for x in sel_col]).show()

+-----+----+
| disp|drat|
+-----+----+
|160.0| 3.9|
|160.0| 3.9|
|108.0|3.85|
|258.0|3.08|
|360.0|3.15|
|225.0|2.76|
|360.0|3.21|
|146.7|3.69|
|140.8|3.92|
|167.6|3.92|
|167.6|3.92|
|275.8|3.07|
|275.8|3.07|
|275.8|3.07|
|472.0|2.93|
|460.0| 3.0|
|440.0|3.23|
| 78.7|4.08|
| 75.7|4.93|
| 71.1|4.22|
+-----+----+
only showing top 20 rows



In [ ]:
mtcars.show

### udf Function
It allows us to transfer a user defined function to a pyspark.sql.functions function which can act on columns of a DataFrame. It makes data framsformation much more flexible.

Using udf() could be tricky. The key to succeed is to understand how to define the returnType parameter.

In [58]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf
mtcars.show(5)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|            model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



In [59]:
def disp_by_hp(disp,hp):
    return(disp/hp)

In [61]:
disp_by_hp_udf=udf(disp_by_hp,returnType=FloatType())

In [63]:
mtcars.select([eval('mtcars.'+x) for x in mtcars.columns] +[disp_by_hp_udf('disp','hp').name('disp/hp')]).show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+---------+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|  disp/hp|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+---------+
|          Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|1.4545455|
|      Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|1.4545455|
|         Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|1.1612903|
|     Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|2.3454545|
|  Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2| 2.057143|
|            Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1| 2.142857|
|         Duster 360|14.3|  8|360.0|245|3.21| 3.57|15.84|  0|  0|   3|   4|1.4693878|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19| 20.0|  1|  0|   4|   2| 2.366129|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22

In [64]:
def merge_two_col(col1, col2):
    return([col1, col2])

In [65]:
array_type=ArrayType(FloatType())

In [66]:
merge_two_col_udf=udf(merge_two_col,returnType=array_type)

In [ ]:
mtcars.select([eval('mtcars.'+ x)for x in mtcars.columns ] +
             [])

#### ArrayType vs. StructType
Both `ArrayType` and `StructType` can be used to build `returnType` for a list. The difference is: 

1. `ArrayType` requires all elements in the list have the same `elementType`, while `StructType` can have different `elementTypes`.
2. `StructType` represents a `Row` object.

In [68]:
struct_type_example=StructType([
    StructField('f1',StringType()),
    StructField('f2',FloatType())
])
struct_merge_two_col_udf=udf(merge_two_col,returnType=struct_type_example)

In [73]:
mtcars_array_type =mtcars.select([eval('mtcars.'+ x)for x in mtcars.columns ] +
             [merge_two_col_udf('mpg','disp').name('merged_col')])
mtcars_struct_type =mtcars.select([eval('mtcars.'+ x)for x in mtcars.columns ] +
             [struct_merge_two_col_udf('mpg','disp').name('merged_col')])


In [77]:
mtcars_array_type.rdd.take(2)

[Row(model='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4, merged_col=[21.0, 160.0]),
 Row(model='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4, merged_col=[21.0, 160.0])]

In [76]:
mtcars_struct_type.rdd.take(2)

[Row(model='Mazda RX4', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.62, qsec=16.46, vs=0, am=1, gear=4, carb=4, merged_col=Row(f1='21.0', f2=160.0)),
 Row(model='Mazda RX4 Wag', mpg=21.0, cyl=6, disp=160.0, hp=110, drat=3.9, wt=2.875, qsec=17.02, vs=0, am=1, gear=4, carb=4, merged_col=Row(f1='21.0', f2=160.0))]

The merged_col in mtcars_struct_type is a Row object, but not in mtcars_array_type

## Pipeline



![spark pipeline](spark-pipeline.png)

#### Example 

We are going to use pipeline to StringIndex columns x1, x2, y1, and y2. Then we OneHotEncode the resulting StringIdexed colu

In [78]:
# Let's create some example dataset
import pandas as pd
pdf = pd.DataFrame({
        'x1': ['a','a','b','b', 'b', 'c'],
        'x2': ['apple', 'orange', 'orange','orange', 'peach', 'peach'],
        'x3': [1, 1, 2, 2, 2, 4],
        'x4': [2.4, 2.5, 3.5, 1.4, 2.1,1.5],
        'y1': [1, 0, 1, 0, 0, 1],
        'y2': ['yes', 'no', 'no', 'yes', 'yes', 'yes']
    })
df = spark.createDataFrame(pdf)
df.show()

+---+------+---+---+---+---+
| x1|    x2| x3| x4| y1| y2|
+---+------+---+---+---+---+
|  a| apple|  1|2.4|  1|yes|
|  a|orange|  1|2.5|  0| no|
|  b|orange|  2|3.5|  1| no|
|  b|orange|  2|1.4|  0|yes|
|  b| peach|  2|2.1|  0|yes|
|  c| peach|  4|1.5|  1|yes|
+---+------+---+---+---+---+



In [79]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [82]:
s_indexer=StringIndexer(inputCol='x1',outputCol='idx_x1')
s_indexer_model=s_indexer.fit(df)
df_s_indexer=s_indexer_model.transform(df)
df_s_indexer.show()

+---+------+---+---+---+---+----------+
| x1|    x2| x3| x4| y1| y2|indexed_x1|
+---+------+---+---+---+---+----------+
|  a| apple|  1|2.4|  1|yes|       1.0|
|  a|orange|  1|2.5|  0| no|       1.0|
|  b|orange|  2|3.5|  1| no|       0.0|
|  b|orange|  2|1.4|  0|yes|       0.0|
|  b| peach|  2|2.1|  0|yes|       0.0|
|  c| peach|  4|1.5|  1|yes|       2.0|
+---+------+---+---+---+---+----------+



In [83]:
si_stage=[StringIndexer(inputCol=c,outputCol='indexed_'+c) for c in ['x1','x2','y1','y2']]
si_stage

[StringIndexer_44ada7f2d417ab5e2b17,
 StringIndexer_47d8bdd11531b8557a01,
 StringIndexer_4066b924acf0a7da6f4e,
 StringIndexer_40de89a317c3c8c2d666]

In [85]:
ohe_stage = [OneHotEncoder(inputCol='indexed_'+c,outputCol='ohe_'+c) for c in ['x1','x2','y1','y2']]
ohe_stage

[OneHotEncoder_4ddda91c113382c968dc,
 OneHotEncoder_460d8ab672f85501f12e,
 OneHotEncoder_4f9a85f4c68bb376de4b,
 OneHotEncoder_40f19d85a85373488688]

In [87]:
all_stages=si_stage+ohe_stage
[type(x) for x in all_stages]

[pyspark.ml.feature.StringIndexer,
 pyspark.ml.feature.StringIndexer,
 pyspark.ml.feature.StringIndexer,
 pyspark.ml.feature.StringIndexer,
 pyspark.ml.feature.OneHotEncoder,
 pyspark.ml.feature.OneHotEncoder,
 pyspark.ml.feature.OneHotEncoder,
 pyspark.ml.feature.OneHotEncoder]

In [89]:
Pipeline(stages=all_stages).fit(df).transform(df).show()

+---+------+---+---+---+---+----------+----------+----------+----------+-------------+-------------+-------------+-------------+
| x1|    x2| x3| x4| y1| y2|indexed_x1|indexed_x2|indexed_y1|indexed_y2|       ohe_x1|       ohe_x2|       ohe_y1|       ohe_y2|
+---+------+---+---+---+---+----------+----------+----------+----------+-------------+-------------+-------------+-------------+
|  a| apple|  1|2.4|  1|yes|       1.0|       2.0|       1.0|       0.0|(2,[1],[1.0])|    (2,[],[])|    (1,[],[])|(1,[0],[1.0])|
|  a|orange|  1|2.5|  0| no|       1.0|       0.0|       0.0|       1.0|(2,[1],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|    (1,[],[])|
|  b|orange|  2|3.5|  1| no|       0.0|       0.0|       1.0|       1.0|(2,[0],[1.0])|(2,[0],[1.0])|    (1,[],[])|    (1,[],[])|
|  b|orange|  2|1.4|  0|yes|       0.0|       0.0|       0.0|       0.0|(2,[0],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
|  b| peach|  2|2.1|  0|yes|       0.0|       1.0|       0.0|       0.0|(2,[0],[1.0])|(2,[1],[1.0

## VectorAssembler

To fit a ML model in pyspark, we need to combine all feature columns into one single column of vectors: the **featuresCol**. The `VectorAssembler` can be used to combine multiple **`OneHotEncoder` columns** and **other continuous variable columns** into one single column.



In [91]:
from pyspark.ml.feature import VectorAssembler
df_new=Pipeline(stages=all_stages).fit(df).transform(df)
df_new.show()

+---+------+---+---+---+---+----------+----------+----------+----------+-------------+-------------+-------------+-------------+
| x1|    x2| x3| x4| y1| y2|indexed_x1|indexed_x2|indexed_y1|indexed_y2|       ohe_x1|       ohe_x2|       ohe_y1|       ohe_y2|
+---+------+---+---+---+---+----------+----------+----------+----------+-------------+-------------+-------------+-------------+
|  a| apple|  1|2.4|  1|yes|       1.0|       2.0|       1.0|       0.0|(2,[1],[1.0])|    (2,[],[])|    (1,[],[])|(1,[0],[1.0])|
|  a|orange|  1|2.5|  0| no|       1.0|       0.0|       0.0|       1.0|(2,[1],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|    (1,[],[])|
|  b|orange|  2|3.5|  1| no|       0.0|       0.0|       1.0|       1.0|(2,[0],[1.0])|(2,[0],[1.0])|    (1,[],[])|    (1,[],[])|
|  b|orange|  2|1.4|  0|yes|       0.0|       0.0|       0.0|       0.0|(2,[0],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|
|  b| peach|  2|2.1|  0|yes|       0.0|       1.0|       0.0|       0.0|(2,[0],[1.0])|(2,[1],[1.0

#### Assembling feature columns into one single feacturesCol with VectorAssembler

In [92]:
df_new.columns

['x1',
 'x2',
 'x3',
 'x4',
 'y1',
 'y2',
 'indexed_x1',
 'indexed_x2',
 'indexed_y1',
 'indexed_y2',
 'ohe_x1',
 'ohe_x2',
 'ohe_y1',
 'ohe_y2']

In [96]:
df_assembled=VectorAssembler(inputCols=['ohe_x1','ohe_x2','ohe_y1','x4'],outputCol= 'featuresCol')\
            .transform(df_new)\
            .drop('indexed_x1','indexed_x2','indexed_y1','indexed_y2')
df_assembled.show(truncate=False)

+---+------+---+---+---+---+-------------+-------------+-------------+-------------+-------------------------+
|x1 |x2    |x3 |x4 |y1 |y2 |ohe_x1       |ohe_x2       |ohe_y1       |ohe_y2       |featuresCol              |
+---+------+---+---+---+---+-------------+-------------+-------------+-------------+-------------------------+
|a  |apple |1  |2.4|1  |yes|(2,[1],[1.0])|(2,[],[])    |(1,[],[])    |(1,[0],[1.0])|(6,[1,5],[1.0,2.4])      |
|a  |orange|1  |2.5|0  |no |(2,[1],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|(1,[],[])    |[0.0,1.0,1.0,0.0,1.0,2.5]|
|b  |orange|2  |3.5|1  |no |(2,[0],[1.0])|(2,[0],[1.0])|(1,[],[])    |(1,[],[])    |[1.0,0.0,1.0,0.0,0.0,3.5]|
|b  |orange|2  |1.4|0  |yes|(2,[0],[1.0])|(2,[0],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|[1.0,0.0,1.0,0.0,1.0,1.4]|
|b  |peach |2  |2.1|0  |yes|(2,[0],[1.0])|(2,[1],[1.0])|(1,[0],[1.0])|(1,[0],[1.0])|[1.0,0.0,0.0,1.0,1.0,2.1]|
|c  |peach |4  |1.5|1  |yes|(2,[],[])    |(2,[1],[1.0])|(1,[],[])    |(1,[0],[1.0])|(6,[3,5],[1.0,1.5])      |
+

## Duplicates, Outliers, Missing observations
Your data can be stained with duplicates, missing observations and outliers, non-existent addresses, wrong phone numbers and area codes, inaccurate geographical coordinates, wrong dates, incorrect labels, mixtures of upper and lower cases, trailing spaces, and many other more subtle problems. It is your job to clean it, irrespective of whether you are a data scientist or data engineer, so you can build a statistical or machine learning model.


### Duplicates

In [97]:
# Example data

df = spark.createDataFrame([
(1, 144.5, 5.9, 33, 'M'),
(2, 167.2, 5.4, 45, 'M'),
(3, 124.1, 5.2, 23, 'F'),
(4, 144.5, 5.9, 33, 'M'),
(5, 133.2, 5.7, 54, 'F'),
(3, 124.1, 5.2, 23, 'F'),
(5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])
df.show()


+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [98]:
df.count()

7

In [99]:
df.distinct().count()

6

In [101]:
df=df.drop_duplicates()
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  5| 129.2|   5.3| 42|     M|
|  1| 144.5|   5.9| 33|     M|
|  4| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
+---+------+------+---+------+



In [104]:
df=df.dropDuplicates(subset=[c for c in df.columns if c!='id'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [106]:
import pyspark.sql.functions as fn
df.agg(
        fn.count('id').alias('count'),
        fn.countDistinct('id').alias('distinct')
).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



In [107]:
df.withColumn('id',fn.monotonically_increasing_id()).show()

+-------------+------+------+---+------+
|           id|weight|height|age|gender|
+-------------+------+------+---+------+
|  25769803776| 133.2|   5.7| 54|     F|
| 171798691840| 144.5|   5.9| 33|     M|
| 592705486848| 167.2|   5.4| 45|     M|
|1236950581248| 124.1|   5.2| 23|     F|
|1365799600128| 129.2|   5.3| 42|     M|
+-------------+------+------+---+------+



### Missing observations

In [3]:
# Example data
df_miss = spark.createDataFrame([
        (1, 143.5, 5.6, 28,   'M',  100000),
        (2, 167.2, 5.4, 45,   'M',  None),
        (3, None , 5.2, None, None, None),
        (4, 144.5, 5.9, 33,   'M',  None),
        (5, 133.2, 5.7, 54,   'F',  None),
        (6, 124.1, 5.2, None, 'F',  None),
        (7, 129.2, 5.3, 42,   'M',  76000),
    ], ['id', 'weight', 'height', 'age', 'gender', 'income'])

df_miss.show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [4]:
df_miss.where('id==3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
+---+------+------+----+------+------+



In [5]:
df_miss.rdd.map(lambda row: (row['id'], sum([c==None for c in row]))).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [6]:
df_miss.dropna(thresh=3).show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [9]:
df_miss_no_income=df_miss.select([c for c in df_miss.columns if c!='income'])
df_miss_no_income.show()


+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



In [ ]:
df_miss.drop

In [11]:
import pyspark.sql.functions as fn
means=df_miss_no_income.agg(*[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c!='gender'])\
        .toPandas().to_dict('records')[0]
print(means)

{'id': 4.0, 'weight': 140.28333333333333, 'height': 5.471428571428571, 'age': 40.4}


In [12]:
means['gender']='missing'
means

{'age': 40.4,
 'gender': 'missing',
 'height': 5.471428571428571,
 'id': 4.0,
 'weight': 140.28333333333333}

In [13]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            weight|height|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



### Outliers

In [14]:
# Example data
df_outliers = spark.createDataFrame([
        (1, 143.5, 5.3, 28),
        (2, 154.2, 5.5, 45),
        (3, 342.3, 5.1, 99),
        (4, 144.5, 5.5, 33),
        (5, 133.2, 5.4, 54),
        (6, 124.1, 5.1, 21),
        (7, 129.2, 5.3, 42),
    ], ['id', 'weight', 'height', 'age'])

In [15]:
df_outliers.show()

+---+------+------+---+
| id|weight|height|age|
+---+------+------+---+
|  1| 143.5|   5.3| 28|
|  2| 154.2|   5.5| 45|
|  3| 342.3|   5.1| 99|
|  4| 144.5|   5.5| 33|
|  5| 133.2|   5.4| 54|
|  6| 124.1|   5.1| 21|
|  7| 129.2|   5.3| 42|
+---+------+------+---+



In [16]:
cols=['weight','age']
bounds={}
for col in cols:
    quantiles=df_outliers.approxQuantile(col,[.25, .75], 0.05)
    IQR=quantiles[1]-quantiles[0]
    bounds[col]=[quantiles[0]-1.5*IQR, quantiles[1]+1.5*IQR]


In [17]:
bounds

{'age': [-11.0, 93.0], 'weight': [91.69999999999999, 191.7]}

In [18]:
quantiles

[28.0, 54.0]

In [20]:
outliers=df_outliers.select(*['id'] +
                           [
                               (
                                   (df_outliers[c]<bounds[c][0]) |
                                   (df_outliers[c]>bounds[c][1])
                               ).alias(c+'_o') for c in cols
                           ]
                           )
outliers.show()

+---+--------+-----+
| id|weight_o|age_o|
+---+--------+-----+
|  1|   false|false|
|  2|   false|false|
|  3|    true| true|
|  4|   false|false|
|  5|   false|false|
|  6|   false|false|
|  7|   false|false|
+---+--------+-----+



In [21]:
df_outliers=df_outliers.join(outliers,on="id")

In [22]:
df_outliers.show()

+---+------+------+---+--------+-----+
| id|weight|height|age|weight_o|age_o|
+---+------+------+---+--------+-----+
|  7| 129.2|   5.3| 42|   false|false|
|  6| 124.1|   5.1| 21|   false|false|
|  5| 133.2|   5.4| 54|   false|false|
|  1| 143.5|   5.3| 28|   false|false|
|  3| 342.3|   5.1| 99|    true| true|
|  2| 154.2|   5.5| 45|   false|false|
|  4| 144.5|   5.5| 33|   false|false|
+---+------+------+---+--------+-----+



In [23]:
df_outliers.filter(df_outliers.age_o==0).show()

+---+------+------+---+--------+-----+
| id|weight|height|age|weight_o|age_o|
+---+------+------+---+--------+-----+
|  7| 129.2|   5.3| 42|   false|false|
|  6| 124.1|   5.1| 21|   false|false|
|  5| 133.2|   5.4| 54|   false|false|
|  1| 143.5|   5.3| 28|   false|false|
|  2| 154.2|   5.5| 45|   false|false|
|  4| 144.5|   5.5| 33|   false|false|
+---+------+------+---+--------+-----+



First, we calculate the lower and upper cut off points for each feature.

## Understand your data

#### Import data

In [24]:
titanic=spark.read.csv('kaggle-titanic-train.csv',header=True,inferSchema=True)
titanic.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [25]:
titanic.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [26]:
titanic.groupby('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



#### Summary statistics

In [27]:
len(titanic.columns)

12

In [28]:
titanic.count()

891

In [32]:
titanic.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [34]:
def describe_columns(df):
    for i in df.columns:
        print('Column: ' + i)
        df.select(i).describe().show()

In [35]:
describe_columns(titanic)

Column: PassengerId
+-------+-----------------+
|summary|      PassengerId|
+-------+-----------------+
|  count|              891|
|   mean|            446.0|
| stddev|257.3538420152301|
|    min|                1|
|    max|              891|
+-------+-----------------+

Column: Survived
+-------+-------------------+
|summary|           Survived|
+-------+-------------------+
|  count|                891|
|   mean| 0.3838383838383838|
| stddev|0.48659245426485753|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

Column: Pclass
+-------+------------------+
|summary|            Pclass|
+-------+------------------+
|  count|               891|
|   mean| 2.308641975308642|
| stddev|0.8360712409770491|
|    min|                 1|
|    max|                 3|
+-------+------------------+

Column: Name
+-------+--------------------+
|summary|                Name|
+-------+--------------------+
|  count|                 891|
|   mean|                

#### Calculate statistics

To calculate a given statistics we can use function agg combined with appropriate aggregation function.  
A list of aggregation functions (the names are fairly self-explanatory) includes:
`avg()`, `count()`, `countDistinct()`, `first()`, `kurtosis()`, `max()`, `mean()`, `min()`,
`skewness()`, `stddev()`, `stddev_pop()`, `stddev_samp()`, `sum()`, `sumDistinct()`,
`var_pop()`, `var_samp()` and `variance()`.

In [36]:
titanic.agg({'Fare': 'skewness'}).show()

+-----------------+
|   skewness(Fare)|
+-----------------+
|4.779253292372357|
+-----------------+



#### Correlations

Calculating correlations in PySpark is very easy once your data is in a DataFrame
form. The only difficulties are that the `.corr(...)` method supports the Pearson
correlation coefficient at the moment, and it can only calculate pairwise correlations,
such as the following:

In [37]:
titanic.corr('Fare','Age')

0.135515853527051